In [8]:
import pandas as pd
import numpy as np
from nltk.corpus import stopwords
stop = stopwords.words('english')
nltk.download('vader_lexicon')
import string
import nltk
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from nltk.corpus import wordnet
nltk.download('averaged_perceptron_tagger')
nltk.download('wordnet')
nltk.download('stopwords')

[nltk_data] Downloading package vader_lexicon to C:\Users\Husein
[nltk_data]     Ghadiali\AppData\Roaming\nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!


In [9]:
# Loading 5 CSV files into 5 dataframes and merging them into one dataframe
df1 = pd.read_csv('review_df_lowes.com_1002969002.csv')
df2 = pd.read_csv('review_df_lowes.com_1002969006.csv')
df3 = pd.read_csv('review_df_lowes.com_5001993533.csv')
df4 = pd.read_csv('review_df_lowes.com_5001993545.csv')
df5 = pd.read_csv('review_df_lowes.com_5003328827.csv')

In [10]:
#Merging all dataframes into one dataframe
df = pd.concat([df1, df2, df3, df4, df5], ignore_index=True)
df.head()


,Market_place,URL,Product_name,SKU,Price,Currency,Total_stars_ranking,Subject,Author,Date,Review,Stars,like,dislike
0,lowes.com,https://www.lowes.com/pd/Mansfield-Summit-Whit...,Mansfield Summit White Elongated Chair Height ...,Model #5384CTK,209.0,USD,4.0,"Good toilet, complete kit w/ soft-close seat",Andrew,"July 12, 2021","Install went mostly according to plan, however...",5,13,2
1,lowes.com,https://www.lowes.com/pd/Mansfield-Summit-Whit...,Mansfield Summit White Elongated Chair Height ...,Model #5384CTK,209.0,USD,4.0,Easy Install,Rusty,"July 3, 2021",This was a very easy install. Make sure that y...,5,6,0
2,lowes.com,https://www.lowes.com/pd/Mansfield-Summit-Whit...,Mansfield Summit White Elongated Chair Height ...,Model #5384CTK,209.0,USD,4.0,Hotel Flusher,Rope cushions,"August 6, 2021",The first thing I want to say is if you've eve...,4,10,1
3,lowes.com,https://www.lowes.com/pd/Mansfield-Summit-Whit...,Mansfield Summit White Elongated Chair Height ...,Model #5384CTK,209.0,USD,4.0,Gorgeous toilet,EricF,"July 17, 2021","When I got the toilet, was thinking it is no d...",5,6,3
4,lowes.com,https://www.lowes.com/pd/Mansfield-Summit-Whit...,Mansfield Summit White Elongated Chair Height ...,Model #5384CTK,209.0,USD,4.0,Great functioning toilet,BigSal1,"August 29, 2021",A toilet is a toilet pretty much. Good things ...,5,8,0


In [11]:
#Checking for null values
df.isnull().sum()

Market_place             0
URL                      0
Product_name             0
SKU                      0
Price                    0
Currency                 0
Total_stars_ranking      0
Subject                296
Author                 166
Date                     0
Review                 282
Stars                    0
like                     0
dislike                  0
dtype: int64

In [12]:
# Dropping null values in the review column
df = df.dropna(subset=['Review'])

In [13]:
# Checking to see if all null values have been dropped from Review column
df.isnull().sum()

Market_place            0
URL                     0
Product_name            0
SKU                     0
Price                   0
Currency                0
Total_stars_ranking     0
Subject                14
Author                  6
Date                    0
Review                  0
Stars                   0
like                    0
dislike                 0
dtype: int64

In [14]:
# Dropping unnecessary columns
df.drop(['Market_place', 'URL', 'Currency', 'Author', 'Total_stars_ranking'], axis=1, inplace=True)
df.head()

,Product_name,SKU,Price,Subject,Date,Review,Stars,like,dislike
0,Mansfield Summit White Elongated Chair Height ...,Model #5384CTK,209.0,"Good toilet, complete kit w/ soft-close seat","July 12, 2021","Install went mostly according to plan, however...",5,13,2
1,Mansfield Summit White Elongated Chair Height ...,Model #5384CTK,209.0,Easy Install,"July 3, 2021",This was a very easy install. Make sure that y...,5,6,0
2,Mansfield Summit White Elongated Chair Height ...,Model #5384CTK,209.0,Hotel Flusher,"August 6, 2021",The first thing I want to say is if you've eve...,4,10,1
3,Mansfield Summit White Elongated Chair Height ...,Model #5384CTK,209.0,Gorgeous toilet,"July 17, 2021","When I got the toilet, was thinking it is no d...",5,6,3
4,Mansfield Summit White Elongated Chair Height ...,Model #5384CTK,209.0,Great functioning toilet,"August 29, 2021",A toilet is a toilet pretty much. Good things ...,5,8,0


### Fixing encoding issues â€™ were showing in excel instead of " ' ". Assuming from webscrapping

In [17]:
# Fix encoding issues in text by re-encoding from Windows-1252 to UTF-8
def fix_encoding(text):
    return text.encode('windows-1252', errors='ignore').decode('utf-8', errors='ignore')

In [18]:
# Apply the fix_encoding function to the Review column
df['Review'] = df['Review'].apply(fix_encoding)

In [20]:
# Map treebank part of speech tags to WordNet part of speech tags
def get_wordnet_pos(treebank_tag):
    if treebank_tag.startswith('J'):
        return wordnet.ADJ
    elif treebank_tag.startswith('V'):
        return wordnet.VERB
    elif treebank_tag.startswith('N'):
        return wordnet.NOUN
    elif treebank_tag.startswith('R'):
        return wordnet.ADV
    else:
        return wordnet.NOUN

In [22]:
# Data preprocessing function
def preprocess_text(text):
    """
    Preprocess text by removing punctuation, stop words, and lemmatizing
    """
    # Remove punctuation
    translator = str.maketrans('', '', string.punctuation)
    text = text.translate(translator)

    # Remove stop words
    stop_words = set(stopwords.words('english'))
    tokens = nltk.word_tokenize(text)
    tokens = [token for token in tokens if token.lower() not in stop_words]
    
    # Lemmatize tokens using their part of speech tags
    lemmatizer = WordNetLemmatizer()
    pos_tags = nltk.pos_tag(tokens)
    lemmatized_tokens = [lemmatizer.lemmatize(token, get_wordnet_pos(pos)) for token, pos in pos_tags]
    
    return ' '.join(lemmatized_tokens)

In [23]:
# Apply the preprocess_text function to the Review column
df['Review'] = df['Review'].apply(preprocess_text)

In [24]:
df.head()

,Product_name,SKU,Price,Subject,Date,Review,Stars,like,dislike
0,Mansfield Summit White Elongated Chair Height ...,Model #5384CTK,209.0,"Good toilet, complete kit w/ soft-close seat","July 12, 2021",Install go mostly accord plan however bolts se...,5,13,2
1,Mansfield Summit White Elongated Chair Height ...,Model #5384CTK,209.0,Easy Install,"July 3, 2021",easy install Make sure someone help carry box ...,5,6,0
2,Mansfield Summit White Elongated Chair Height ...,Model #5384CTK,209.0,Hotel Flusher,"August 6, 2021",first thing want say youve ever stayed hotel u...,4,10,1
3,Mansfield Summit White Elongated Chair Height ...,Model #5384CTK,209.0,Gorgeous toilet,"July 17, 2021",get toilet think different get installed appar...,5,6,3
4,Mansfield Summit White Elongated Chair Height ...,Model #5384CTK,209.0,Great functioning toilet,"August 29, 2021",toilet toilet pretty much Good thing toilet po...,5,8,0


In [26]:
# Save to CSV file
df.to_csv('master_review_v2.csv', index=False)